<a href="https://colab.research.google.com/github/dmkolesova/SE_SQL/blob/main/%22DB_INSERT_ANY_TYPE_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Код для генерации данных PostgreSQL

Инструкция по генерации строк:
1. Все ячейки кода необходимо запустить в той последовательности, в который они представлены
2. Для генерации данных для своего кейса необходимо перейти на ячейку (ссылка) и ввести свои поля

### 1. Установка библиотек и импорт модулей

In [2]:
!pip install Faker

import random
import datetime
from faker import Faker
from google.colab import files

from faker.providers.address.ru_RU import Provider

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.0 MB/s eta 0:00:00


### 2. Формирование функций генерации

In [60]:
fake = Faker('ru_RU')
fake.add_provider(Provider)

# def generate_rows(*, row_number, table_name, column_names):
#   query = "";
#   columns_str = ', '.join(f'"{column}"' for column in column_names)
#   for i in range(row_number):
#         # генерируем значения для каждого столбца
#         name=f"'{fake.unique.name()}'"
#         email=f"'{fake.unique.email()}'"
#         phone=f"'{fake.phone_number()}'"
#         state=fake.random_int(min=1, max=4)

#         # формируем список со значениями
#         values = [i+1, name, email, phone, state]
#         # формируем строку со значениями
#         values_str = ', '.join(str(value) for value in values)
#         # формируем DML-запрос
#         insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
#         query += insert_query
#   return query

def get_fake_type(*, type):
  start_date = datetime.date(2023, 1, 1)
  end_date = datetime.date(2023, 12, 31)
  if type == 'name':
    return f"'{fake.unique.name()}'"
  elif type == 'email':
    return f"'{fake.unique.email()}'"
  elif type == 'phone':
    return f"'{fake.phone_number()}'"
  elif type == 'company':
    return f"'{fake.company()}'"
  elif type == 'address':
    return f"'{fake.address()}'"
  elif type.startswith('fk_int'):
    return fake.random_int(min=int(str.split(type, '_')[-2]), max=int(str.split(type, '_')[-1]))
  elif type == 'float':
    return round(fake.pyfloat(right_digits=2, min_value=0, max_value=10000), 2)
  elif type == 'date':
    return f"'{(start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))).strftime('%Y-%m-%d')}'"
  else:
    return 'Incorrect type'

def generate_rows(*, row_number, table_name, column_names):
  query = "";
  columns_str = ', '.join(f'"{column}"' for column in column_names.keys())
  for i in range(row_number):
    values = list()
    values.append(i+1)
    for k, v in column_names.items():
      if v != 'id':
        values.append(get_fake_type(type=v))
    values_str = ', '.join(str(value) for value in values)
    insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
    query += insert_query
  return query


def save_to_file(*, row_number, table_name, column_names):
  filename = f'{table_name}.sql'
  with open(filename, 'w') as f:
    generated_rows = generate_rows(row_number=row_number, table_name=table_name, column_names=column_names)
    f.write(generated_rows)
  f.close()
  files.download(filename)

### 3. Заполнение переменных

> Название таблицы

In [61]:
table_name = 'customer'

> Количество строк для генерации

In [62]:
k = 5

> Столбцы таблицы

Правила оформления:
1. Составляется ввиде словаря {"k":"v"}, где "k" - название поля, "v" - тип поля.
2. Обязательно наличие поля с типом "id"
3. Тип поля может быть одним из:


* name - ФИО, например Иван Иванович Иванов
* email - почта, например 123@example.org
* phone - телефон, например 89999011121 или +7 (999) 023-4444
* company - наименование компаний, например ОАО Ромашка
* address - адрес, например с. Владивосток, алл. Запорожская, д. 1 к. 1, 111111
* date - дата в диапазоне от 2023-01-01 до 2023-12-31
* float - число с плавающей точкой, например 78.73
* fk_int_0_100 - поле FOREIGN KEY, где ключ, на который ссылается должен быть типа int. При этом диапазон значений (0_100) может быть изменен в соответсвии с вашим кейсом. Например, fk_int_1_5 или fk_int_0_34568



In [63]:
column_names = {'id':'id', 'name':'name', 'phone': 'phone', 'state': 'fk_int_0_5', 'cmp': 'company', 'addr': 'address', 'price': 'float', 'dt':'date', 'e':'email'}

### 4. Пример генерации записей с выводом на экран

In [64]:
q = generate_rows(row_number=k, table_name=table_name, column_names=column_names)
print(q)

INSERT INTO "customer" ("id", "name", "phone", "state", "cmp", "addr", "price", "dt", "e") VALUES (1, 'Григорьев Эраст Ефимьевич', '8 156 552 07 49', 2, 'ГК ТНС энерго', 'ст. Верхний Тагил, ш. Приморское, д. 97 к. 6/7, 023479', 95.48, '2023-01-15', 'milannikolaev@example.net');
INSERT INTO "customer" ("id", "name", "phone", "state", "cmp", "addr", "price", "dt", "e") VALUES (2, 'Афанасий Изотович Тарасов', '8 343 332 0441', 0, 'НПО «Константинова, Галкин и Кудрявцев»', 'г. Кировск (Ленин.), пр. Декабристов, д. 8, 435042', 62.08, '2023-01-15', 'gennadi1989@example.com');
INSERT INTO "customer" ("id", "name", "phone", "state", "cmp", "addr", "price", "dt", "e") VALUES (3, 'Авдеева Любовь Тимофеевна', '8 (572) 011-0850', 0, 'Роснефть', 'с. Учалы, пр. З.Космодемьянской, д. 226 к. 4, 268165', 34.68, '2023-08-11', 'martjan69@example.net');
INSERT INTO "customer" ("id", "name", "phone", "state", "cmp", "addr", "price", "dt", "e") VALUES (4, 'Филипп Жанович Шарапов', '8 218 852 27 17', 4, 'ООО

### 5. Пример с генерацией записей в файл

In [65]:
save_to_file(row_number=k, table_name=table_name, column_names=column_names)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [46]:
column_names = {'id':'id', 'name':'name', 'phone': 'phone', 'state': 'fk_int_0_100', 'cmp': 'company', 'addr': 'address', 'price': 'float', 'dt':'date'}



query = "";
columns_str = ', '.join(f'"{column}"' for column in column_names.keys())


# for key, value in column_names.items():
  # print(key)
  # print(value)
  # print(get_fake_type(type=value))



for i in range(5):
  values = list()
  values.append(i+1)
  for k, v in column_names.items():
    # print(k)
    if v != 'id':
      values.append(get_fake_type(type=v))
  values_str = ', '.join(str(value) for value in values)
  insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
  # print(insert_query)
  query += insert_query
print(query)

INSERT INTO "customer" ("id", "name", "phone", "state", "cmp", "addr", "price", "dt") VALUES (1, 'Кузьма Эдгардович Наумов', '8 (344) 533-9673', 49, 'ЗАО «Мишин»', 'с. Владивосток, алл. Запорожская, д. 5 к. 6, 546658', 78.73, '2023-05-13');
INSERT INTO "customer" ("id", "name", "phone", "state", "cmp", "addr", "price", "dt") VALUES (2, 'Тарасов Евстигней Арсенович', '+7 (382) 797-5720', 62, 'НПО «Силин»', 'ст. Нерчинск, ш. Черемушки, д. 8/7 к. 65, 929400', 36.28, '2023-08-18');
INSERT INTO "customer" ("id", "name", "phone", "state", "cmp", "addr", "price", "dt") VALUES (3, 'Романов Капитон Игнатьевич', '+7 (404) 864-37-17', 8, 'Воронцова Лимитед', 'к. Бавлы, наб. Песчаная, д. 9/6 к. 5/2, 021070', 65.31, '2023-11-06');
INSERT INTO "customer" ("id", "name", "phone", "state", "cmp", "addr", "price", "dt") VALUES (4, 'Ефремов Аскольд Евсеевич', '83595460209', 81, 'ООО «Мамонтова Виноградов»', 'ст. Павловская, пр. Инженерный, д. 248, 784798', 2.1, '2023-03-05');
INSERT INTO "customer" ("id"

In [15]:
# generate_rows(k, table_name, column_names)
q = generate_rows(row_number=k, table_name=table_name, column_names=column_names)
print(q)

INSERT INTO "customer" ("id", "name", "email", "phone", "state") VALUES (1, 'Доброслав Гурьевич Меркушев', 'andreevamos@example.com', '8 (740) 601-4521', 1);
INSERT INTO "customer" ("id", "name", "email", "phone", "state") VALUES (2, 'Михайлова Вероника Григорьевна', 'viktorin_1983@example.net', '+7 (630) 074-66-91', 3);
INSERT INTO "customer" ("id", "name", "email", "phone", "state") VALUES (3, 'Васильев Павел Анатольевич', 'arhipovmir@example.net', '+7 420 132 91 95', 3);
INSERT INTO "customer" ("id", "name", "email", "phone", "state") VALUES (4, 'Воронцова Дарья Тимуровна', 'raisa41@example.net', '8 315 709 5460', 3);
INSERT INTO "customer" ("id", "name", "email", "phone", "state") VALUES (5, 'Бирюкова Зоя Рудольфовна', 'osip_1987@example.net', '+7 877 241 9838', 1);



In [ ]:
# Создание объекта Faker с локализацией для России
fake = Faker('ru_RU')
fake.add_provider(Provider)

# генерация и запись DML-скрипта в файл
# filename = f'{table_name}.sql' # формируем название выходного файла
# with open(filename, 'w') as f: #открытие файла для записи
    # формируем строку с названиями столбцов
columns_str = ', '.join(f'"{column}"' for column in column_names)
for i in range(k):
        # генерируем значения для каждого столбца
        #для строковых значений обязательно f"'{}'"
        #для целочисленных значений используем fake.random_int(min=1, max=4), в скобках указываем диапазон от мин до макс
        name=f"'{fake.unique.name()}'" #генерируем случайное уникальное имя
        email=f"'{fake.unique.email()}'" #генерируем случайный уникальный емейл
        phone=f"'{fake.phone_number()}'" #генерируем случайный номер телефона
        state=fake.random_int(min=1, max=4) #генерируем случайное значение внешнего ключа

        # формируем список со значениями
        values = [i+1, name, email, phone, state]
        # формируем строку со значениями
        values_str = ', '.join(str(value) for value in values)
        # формируем и записываем DML-запрос в файл
        insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
        print(insert_query) #вывод на консоль - для проверки и отладки
        # f.write(insert_query) #запись в файл
    # f.close() #закрытие файла

# Скачиваем файл на локальную машину
# files.download(filename)

In [ ]:
#скрипт наполнения случайными данными таблицы фактов с несколькими внешними ключами
#для таблицы delivery
# Создание объекта Faker с локализацией для России
fake = Faker('ru_RU')
fake.add_provider(Provider)

# задаем названиие таблицы
table_name = 'delivery'

# задаем столбцы таблицы
column_names = ('id', 'courier', 'price', 'wieght', 'category', 'sender', 'receiver', 'date', 'state')

# задаем количество строк таблицы, которое хотим заполнить
k = 100

# генерация и запись DML-скрипта в файл
filename = f'{table_name}.sql' # формируем название выходного файла
with open(filename, 'w') as f: #открытие файла для записи
    # формируем строку с названиями столбцов
    columns_str = ', '.join(f'"{column}"' for column in column_names)
    for i in range(k):
        # генерируем значения для каждого столбца
        #для целочисленных значений используем fake.random_int(min=1, max=4), в скобках указываем диапазон от мин до макс
        courier=fake.random_int(min=1, max=100) #генерируем случайное значение внешнего ключа
        price=round(fake.pyfloat(right_digits=2, min_value=0, max_value=10000), 2) #генерируем для суммы денег за заказ случайное вещественное число от 0 до 10000 с двумя знаками после запятой
        wieght=round(fake.pyfloat(right_digits=2, min_value=0, max_value=10000), 2) #генерируем для суммы денег за заказ случайное вещественное число от 0 до 10000 с двумя знаками после запятой
        category=fake.random_int(min=1, max=4) #генерируем случайное значение внешнего ключа
        sender=fake.random_int(min=1, max=100) #генерируем случайное значение внешнего ключа
        receiver=fake.random_int(min=1, max=100) #генерируем случайное значение внешнего ключа

        # генерируем случайную дату в диапазоне
        start_date = datetime.date(2023, 1, 1)
        end_date = datetime.date(2023, 12, 31)
        random_date = start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))
        # преобразуем дату в строку в формате "год-месяц-день"
        #для строковых значений обязательно f"'{}'"
        date = f"'{random_date.strftime('%Y-%m-%d')}'"
        state=fake.random_int(min=1, max=4) #генерируем случайное значение внешнего ключа

        # формируем список со значениями
        values = [i+1, courier, price, wieght, category, sender, receiver, date, state]
        # формируем строку со значениями
        values_str = ', '.join(str(value) for value in values)
        # формируем и записываем DML-запрос в файл
        insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
        print(insert_query) #вывод на консоль - для проверки и отладки
        f.write(insert_query) #запись в файл
    f.close() #закрытие файла

# Скачиваем файл на локальную машину
files.download(filename)

In [ ]:
#скрипт наполнения случайными данными таблицы фактов с несколькими внешними ключами

# Создание объекта Faker с локализацией для России
fake = Faker('ru_RU')
fake.add_provider(Provider)

# задаем названиие таблицы
table_name = 'orders'

# задаем столбцы таблицы
column_names = ('id', 'customer', 'state', 'delivery', 'sum', 'date')

# задаем количество строк таблицы, которое хотим заполнить
k = 100

# генерация и запись DML-скрипта в файл
filename = f'{table_name}.sql' # формируем название выходного файла
with open(filename, 'w') as f: #открытие файла для записи
    # формируем строку с названиями столбцов
    columns_str = ', '.join(f'"{column}"' for column in column_names)
    for i in range(k):
        # генерируем значения для каждого столбца
        #для целочисленных значений используем fake.random_int(min=1, max=4), в скобках указываем диапазон от мин до макс
        customer=fake.random_int(min=1, max=100) #генерируем случайное значение внешнего ключа
        state=fake.random_int(min=1, max=5) #генерируем случайное значение внешнего ключа
        sum=round(fake.pyfloat(right_digits=2, min_value=0, max_value=10000), 2) #генерируем для суммы денег за заказ случайное вещественное число от 0 до 10000 с двумя знаками после запятой
        delivery=fake.random_int(min=1, max=5) #генерируем случайное значение внешнего ключа

        # генерируем случайную дату в диапазоне
        start_date = datetime.date(2023, 1, 1)
        end_date = datetime.date(2023, 12, 31)
        random_date = start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))
        # преобразуем дату в строку в формате "год-месяц-день"
        #для строковых значений обязательно f"'{}'"
        date = f"'{random_date.strftime('%Y-%m-%d')}'"

        # формируем список со значениями
        values = [i+1, customer, state, delivery, sum, date]
        # формируем строку со значениями
        values_str = ', '.join(str(value) for value in values)
        # формируем и записываем DML-запрос в файл
        insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
        print(insert_query) #вывод на консоль - для проверки и отладки
        f.write(insert_query) #запись в файл
    f.close() #закрытие файла

# Скачиваем файл на локальную машину
files.download(filename)

In [ ]:
#скрипт наполнения случайными данными таблицы фактов с несколькими внешними ключами

# Создание объекта Faker с локализацией для России
fake = Faker('ru_RU')
fake.add_provider(Provider)

# задаем названиие таблицы
table_name = 'order_product'

# задаем столбцы таблицы
column_names = ('id', 'order', 'product', 'quantity')

# задаем количество строк таблицы, которое хотим заполнить
k = 100

# генерация и запись DML-скрипта в файл
filename = f'{table_name}.sql' # формируем название выходного файла
with open(filename, 'w') as f: #открытие файла для записи
    # формируем строку с названиями столбцов
    columns_str = ', '.join(f'"{column}"' for column in column_names)
    for i in range(k):
        # генерируем значения для каждого столбца
        #для целочисленных значений используем fake.random_int(min=1, max=4), в скобках указываем диапазон от мин до макс
        product=fake.random_int(min=1, max=71) #генерируем случайное значение внешнего ключа
        order=fake.random_int(min=1, max=100) #генерируем случайное значение внешнего ключа
        quantity=fake.random_int(min=1, max=15) #генерируем случайное значение количества товара в заказе

        # формируем список со значениями
        values = [i+1, order, product, quantity]
        # формируем строку со значениями
        values_str = ', '.join(str(value) for value in values)
        # формируем и записываем DML-запрос в файл
        insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
        print(insert_query) #вывод на консоль - для проверки и отладки
        f.write(insert_query) #запись в файл
    f.close() #закрытие файла

# Скачиваем файл на локальную машину
files.download(filename)

In [ ]:
#скрипт наполнения случайными данными таблицы с датой и адресом

# Создание объекта Faker с локализацией для России
fake = Faker('ru_RU')
fake.add_provider(Provider)

# задаем названиие таблицы
table_name = 'delivery'

# задаем столбцы таблицы
column_names = ('id', 'date', 'address', 'price')

# задаем количество строк таблицы, которое хотим заполнить
k = 100

# генерация и запись DML-скрипта в файл
filename = f'{table_name}.sql' # формируем название выходного файла
with open(filename, 'w') as f: #открытие файла для записи
    # формируем строку с названиями столбцов
    columns_str = ', '.join(f'"{column}"' for column in column_names)
    for i in range(k):
      # генерируем значения для каждого столбца

        # генерируем случайную дату в диапазоне
        start_date = datetime.date(2023, 1, 1)
        end_date = datetime.date(2023, 12, 31)
        random_date = start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))
        # преобразуем дату в строку в формате "год-месяц-день"
        #для строковых значений обязательно f"'{}'"
        date = f"'{random_date.strftime('%Y-%m-%d')}'"

        #генерируем случайное значение адреса
        address=f"'{fake.address()}'"
        #генерируем для суммы денег за доставку случайное вещественное число от 0 до 1000 с двумя знаками после запятой
        price=round(fake.pyfloat(right_digits=2, min_value=0, max_value=1000), 2)

        # формируем список со значениями
        values = [i+1, date, address, price]
        # формируем строку со значениями
        values_str = ', '.join(str(value) for value in values)
        # формируем и записываем DML-запрос в файл
        insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
        print(insert_query) #вывод на консоль - для проверки и отладки
        f.write(insert_query) #запись в файл
    f.close() #закрытие файла

# Скачиваем файл на локальную машину
files.download(filename)

In [ ]:
#скрипт наполнения случайными данными таблицы с адресом

# Создание объекта Faker с локализацией для России
fake = Faker('ru_RU')
fake.add_provider(Provider)

# задаем названиие таблицы
table_name = 'provider'

# задаем столбцы таблицы
column_names = ('id', 'name', 'phone','email', 'address')

# задаем количество строк таблицы, которое хотим заполнить
k = 100

# генерация и запись DML-скрипта в файл
filename = f'{table_name}.sql' # формируем название выходного файла
with open(filename, 'w') as f: #открытие файла для записи
    # формируем строку с названиями столбцов
    columns_str = ', '.join(f'"{column}"' for column in column_names)
    for i in range(k):
      # генерируем значения для каждого столбца
        #для строковых значений обязательно f"'{}'"
        name=f"'{fake.unique.company()}'" #генерируем случайное уникальное имя
        phone=f"'{fake.phone_number()}'" #генерируем случайный номер телефона
        email=f"'{fake.email()}'" #генерируем случайный уникальный емейл
        address=f"'{fake.address()}'" #генерируем случайное значение адреса

        # формируем список со значениями
        values = [i+1, name, phone, email, address]
        # формируем строку со значениями
        values_str = ', '.join(str(value) for value in values)
        # формируем и записываем DML-запрос в файл
        insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
        print(insert_query) #вывод на консоль - для проверки и отладки
        f.write(insert_query) #запись в файл
    f.close() #закрытие файла

# Скачиваем файл на локальную машину
files.download(filename)

In [ ]:
#скрипт наполнения таблицы-справочника из ограниченного количества значений и ссылкой на внешний ключ
# задаем название таблицы
table_name = 'product'

# задаем столбцы таблицы
column_names = ('id', 'name', 'provider', 'price', 'quantity')

#создаем список данных - справочник значений
products = ['яблоки желтые', 'малина', 'вода', 'хлеб белый','хлеб серый', 'креветки', 'форель', 'апельсины', 'кета','курица','яйцо перепелиное','яйцо куриное','лаваш',
            'булка сдобная','булка сахарная','помидоры бакинские','помидоры чери','огурцы','перец сладкий','перец острый','перец болгарский','мандарины','укроп свежий',
            'укроп сушеный','клубника свежая','клубника мороженная','мороженое','картошка','морковь', 'свекла','пангасиус','семга','кальмар замороженный','горошек зеленый',
            'смородина черная','смородина красная','соль поваренная пищевая йодированная','чай черный байховый','чай зеленый','чай красный','кофе','кофе с молоком','какао',
            'молоко','кефир','сыр с плесенью','сыр плавленый','сыр твердый','сыр мягкий','яблоки красные','яблоки зеленые','яблоки сушеные','икра красная',
            'икра черная','икра заморская баклажанная','масло сливочное','масло оливковое','масло подсолнечное','масло кокосовое','орех грецкий','орех бразильский',
            'лист лавровый','куркума','кукуруза','печенье сладкое','пряники сдобные','тесто слоеное','варенц','ряженка','снежок','шоколад молочный']

# задаем количество строк таблицы, которое хотим заполнить - для таблицы-справочника это количество значений списка данных
k = len(products)

# генерация и запись DML-скрипта в файл
filename = f'{table_name}.sql' # формируем название выходного файла
with open(filename, 'w') as f: #открытие файла для записи
    # формируем строку с названиями столбцов
    columns_str = ', '.join(f'"{column}"' for column in column_names)
    for i in range(k):
        # генерируем значения для каждого столбца
        name=f"'{random.choice(products)}'" #выбираем случайный товар из ограниченного списка
        provider=fake.random_int(min=1, max=100) #генерируем случайное значение внешнего ключа
        price=round(fake.pyfloat(right_digits=2, min_value=0, max_value=3000), 2) #генерируем для суммы денег за товар случайное вещественное число от 0 до 3000 с двумя знаками после запятой
        quantity=fake.random_int(min=1, max=100) #генерируем случайное значение количества товара (на складе))
        # формируем список со значениями
        values = [i+1, name, provider, price, quantity]
        # формируем строку со значениями
        values_str = ', '.join(str(value) for value in values)
        # формируем и записываем DML-запрос в файл
        insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
        print(insert_query) #вывод на консоль - для проверки и отладки
        f.write(insert_query) #запись в файл
    f.close() #закрытие файла

# Скачиваем файл на локальную машину
files.download(filename)

посмотреть, зачем этот скрипт

In [ ]:
from faker import Faker
import re
import random

# Читаем схему данных из файла
with open('/content/sample_data/DDL_PSQL.txt') as f:
    schema = f.read()

# Извлекаем информацию о таблицах и столбцах
table_columns = {}

# Разбиваем схему данных на отдельные запросы
queries = schema.split(';')

for query in queries:
    # Ищем запросы на создание таблиц
    if query.startswith('CREATE TABLE'):
        print("Найден запрос на создание таблицы:", query)
        # Извлекаем имя таблицы и определения столбцов
        table_name = query.split()[2]
        print('table_name ', table_name)

        # Извлекаем определения столбцов
        columns_str = query.split('(')[1].split(')')[0]
        print('columns_str ', columns_str)
        # Разбиваем определения столбцов на отдельные токены
        column_tokens = [token.strip() for token in columns_str.split(',')]
        print('column_tokens ', column_tokens)
        # Извлекаем имена столбцов
        column_names = [token.split()[0] for token in column_tokens]
        print('column_names ', column_names)
        table_columns[table_name] = column_names
        print(table_columns[table_name])

# Заполняем таблицы данными
fake = Faker()

for table_name, columns in table_columns.items():
    for i in range(10):
        values = []
        for column in columns:
            if column == 'id':
                values.append(str(random.randint(1, 100)))
            elif column == 'name':
                values.append(fake.name())
            elif column == 'address':
                values.append(fake.address())
            else:
                values.append(fake.text(max_nb_chars=255))
        print(values)
        placeholders = ','.join(['%s' for _ in columns])
        insert_query = f'INSERT INTO "{table_name}" ("{",".join(columns)}") VALUES ({placeholders});\n'
        print(insert_query)